<a href="https://colab.research.google.com/github/luke-scot/emissions-tracking/blob/main/NMC_flows.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
%%capture
"""Installation and downloads"""
# Install floweaver and display widget packages
%pip install floweaver ipysankeywidget

# Import necessary packages
import pandas as pd
import numpy as np
from floweaver import *
from ipysankeywidget import SankeyWidget

In [149]:
"""Display setup"""
# Enable widget display for Sankeys in Colab
from google.colab import output
output.enable_custom_widget_manager()

In [161]:
flows = pd.read_csv('flows.csv', usecols = ['source', 'target', 'type', 'value'], skipinitialspace=True)
flows.source = flows.source.str.strip()

In [163]:
flows.source.unique()

array(['AM', 'graphite', 'binder', 'copper', 'aluminium', 'electrolyte',
       'plastic', 'steel', 'other', 'nickel', 'cobalt', 'manganese',
       'lithium', 'sodium', 'ammonium', 'precipitation', 'nickel_mining',
       'nickel_refining', 'manganese_mining', 'manganese_refining',
       'cobalt_mining', 'cobalt_refining', 'lithium_mining',
       'lithium_refining', 'graphite_pro', 'aluminium_mining',
       'aluminium_refining', 'aluminium_primary', 'copper_mining',
       'copper_smelting', 'copper_refining', 'Indonesia', 'Philippines',
       'Russia', 'New Caledonia', 'Canada', 'Australia', 'China',
       'Brazil', 'Guatemala', 'South Africa', 'Gabon', 'Ghana', 'India',
       'Ukraine', 'Ivory Coast', 'Malaysia', 'Congo', 'Cuba',
       'Madagascar', 'Papua New Guinea', 'Chile', 'Argentina', 'Guinea',
       'Jamaica', 'Peru', 'Congo, Democratic Republic', 'USA', 'Zambia',
       'Mexico', 'Kazakhstan', 'Poland', 'Mozambique',
       'Korea, Dem. P.R. of'], dtype=object)

In [164]:
flows.target.unique()

array(['NMC622', 'AM', 'nickel_refining', 'nickel', 'manganese_refining',
       'manganese', 'cobalt_refining', 'cobalt', 'lithium_refining',
       'lithium', 'graphite', 'aluminium_refining', 'aluminium_primary',
       'aluminium', 'copper_smelting', 'copper_refining', 'copper',
       'nickel_mining', 'manganese_mining', 'cobalt_mining',
       'lithium_mining', 'aluminium_mining', 'copper_mining',
       'graphite_pro'], dtype=object)

In [165]:
sources =  np.sort(['Indonesia', 'Philippines',
       'Russia', 'New Caledonia', 'Canada', 'Australia', 'China',
       'Brazil', 'Guatemala', 'South Africa', 'Gabon', 'Ghana', 'India',
       'Ukraine', 'Ivory Coast', 'Malaysia', 'Congo', 'Cuba',
       'Madagascar', 'Papua New Guinea', 'Chile', 'Argentina', 'Guinea',
       'Jamaica', 'Peru', 'Congo, Democratic Republic', 'USA', 'Zambia',
       'Mexico', 'Kazakhstan', 'Poland', 'Mozambique',
       'Korea, Dem. P.R. of'])
uses = ['NMC622']
"""Get all unique types of sources and targets listed in products and flows respectively"""
#mining = 
idColumn = np.concatenate((sources,uses))
typeColumn = ['source']*len(sources)+['use']*len(uses)
processes = pd.DataFrame(np.array([idColumn,typeColumn]).transpose(), columns=['id','type'])

In [166]:
dataset = Dataset(flows, dim_process=processes.set_index('id'))

In [167]:
nodes = {
    'sources': ProcessGroup('type == "source"', Partition.Simple('process', sources), title='Sources'),
    # mining
    'nickel_mining': ProcessGroup(['nickel_mining'], title ='Nickel mining'),
    'manganese_mining': ProcessGroup(['manganese_mining'], title ='Manganese mining'),
    'cobalt_mining': ProcessGroup(['cobalt_mining'], title ='Cobalt mining'),
    'lithium_mining': ProcessGroup(['lithium_mining'], title ='Lithium mining'),
    'graphite_pro': ProcessGroup(['graphite_pro'], title ='Graphite production'),
    'aluminium_mining': ProcessGroup(['aluminium_mining'], title ='Aluminium mining'),
    'copper_mining': ProcessGroup(['copper_mining'], title ='Copper mining'),

    # smelting
    'copper_smelting': ProcessGroup(['copper_smelting'], title ='Copper_smelting'),

    # refining
    'nickel_refining': ProcessGroup(['nickel_refining'], title ='Nickel refining'),
    'manganese_refining': ProcessGroup(['manganese_refining'], title ='Manganese refining'),
    'cobalt_refining': ProcessGroup(['cobalt_refining'], title ='Cobalt refining'),
    'lithium_refining': ProcessGroup(['lithium_refining'], title ='Lithium refining'),
    'aluminium_refining': ProcessGroup(['aluminium_refining'], title ='Aluminium refining'),
    'copper_refining': ProcessGroup(['copper_refining'], title ='Copper_refining'),

    # primary
    'aluminium_primary': ProcessGroup(['aluminium_primary'], title ='Aluminium primary'),

    # material
    'sodium': ProcessGroup(['sodium'], title ='Sodium Hydroxide'),
    'ammonium': ProcessGroup(['ammonium'], title ='Ammonium Hydroxide'),
    'precipitation': ProcessGroup(['precipitation'], title ='Precursor Co-precipitation'),
    'nickel': ProcessGroup(['nickel'], title ='Nickel'),
    'manganese': ProcessGroup(['manganese'], title ='Manganese'),
    'cobalt': ProcessGroup(['cobalt'], title ='Cobalt'),
    'lithium': ProcessGroup(['lithium'], title ='Lithium'),
    'AM': ProcessGroup(['AM'], title ='Active material'),
    'aluminium': ProcessGroup(['aluminium'], title ='Aluminium'),
    'graphite': ProcessGroup(['graphite'], title ='Graphite/Carbon'),
    'copper': ProcessGroup(['copper'], title ='Copper'),
    'binder': ProcessGroup(['binder'], title ='Binder'),
    'electrolyte': ProcessGroup(['electrolyte'], title ='Electrolyte'),
    'plastic': ProcessGroup(['plastic'], title ='Plastic'),
    'steel': ProcessGroup(['steel'], title ='Steel'),
    'other': ProcessGroup(['other'], title ='Other'),
    

    # Product
    'NMC622': ProcessGroup('type == "use"', Partition.Simple('process', uses), title='Product'),

}

In [168]:
ordering = [
    [['sources'],[]],
    [['aluminium_mining', 'graphite_pro', 'copper_mining', 'cobalt_mining', 'lithium_mining', 'manganese_mining','nickel_mining'],[]],
    [['copper_smelting'],[]],
    [['aluminium_refining','copper_refining', 'cobalt_refining','lithium_refining','manganese_refining','nickel_refining'],[]],
    [['aluminium_primary','cobalt','lithium','manganese','nickel'],['sodium','ammonium','precipitation']],
    [['aluminium','graphite','copper','AM'], ['binder','electrolyte','plastic','steel','other']],
    [['NMC622'],[]],
]

In [169]:
bundles = [
    # Source to mining
    Bundle("sources","nickel_mining"),  
    Bundle("sources","manganese_mining"),  
    Bundle("sources","cobalt_mining"),   
    Bundle("sources","lithium_mining"),    
    Bundle("sources","graphite_pro"),  
    Bundle("sources","aluminium_mining"),  
    Bundle("sources","copper_mining"),  
    # Mining to refining
    Bundle("copper_mining", "copper_smelting"),
    Bundle("copper_smelting", "copper_refining"), 
    Bundle("nickel_mining", "nickel_refining"),  
    Bundle("manganese_mining", "manganese_refining"),
    Bundle("cobalt_mining", "cobalt_refining"),  
    Bundle("lithium_mining", "lithium_refining"),  
    Bundle("aluminium_mining", "aluminium_refining"), 

    # Refining to material
    Bundle("nickel_refining", "nickel"), 
    Bundle("manganese_refining", "manganese"),
    Bundle("cobalt_refining", "cobalt"), 
    Bundle("lithium_refining", "lithium"),
    Bundle("nickel", "AM"), 
    Bundle("manganese", "AM"),
    Bundle("cobalt", "AM"), 
    Bundle("lithium", "AM"),
    Bundle("sodium", "AM"),
    Bundle("ammonium", "AM"),
    Bundle("precipitation", "AM"),
    Bundle("aluminium_refining", "aluminium_primary"),
    Bundle("aluminium_primary", "aluminium"), 
    Bundle("graphite_pro", "graphite"),
    Bundle("copper_refining", "copper"),

    # Material to product
    Bundle("AM", "NMC622"),      
    Bundle("graphite", "NMC622"),
    Bundle("aluminium", "NMC622"),
    Bundle("copper", "NMC622"),
    Bundle("binder", "NMC622"),
    Bundle("electrolyte", "NMC622"),  
    Bundle("plastic", "NMC622"),  
    Bundle("steel", "NMC622"),  
    Bundle("other", "NMC622"),         
]

In [172]:
flow_partition = Partition.Simple("type", np.unique(flows['type'].to_list()))

sdd = SankeyDefinition(nodes, bundles, ordering, flow_partition = flow_partition)
weave(sdd, dataset, palette='Paired_12').to_widget(width=1000, height = 800)\
.auto_save_svg('NMC622_sankey.svg')

SankeyWidget(groups=[{'id': 'sources', 'type': 'process', 'title': 'Sources', 'nodes': ['sources^Argentina', '…